In [2]:
import numpy as np
import pickle

## Load Data

In [20]:
data = np.zeros([32,40,40,8064])
labels = np.zeros([32,40,4])

for i in range(1, 33):
    if i < 10:
        file = "./data_preprocessed_python/s{0}.dat".format("0" + str(i))
    else:
        file = "./data_preprocessed_python/s{0}.dat".format(str(i))
    with open(file, 'rb') as f: 
        content = pickle.load(f, encoding= "latin1")
    data[i-1,:,:,:] = content["data"]
    labels[i-1,:,:] = content["labels"]

In [21]:
# 32 subjects that watched 40 videos each with 40 sensors (32 EEG + peripherals) for 60s
data.shape

(32, 40, 40, 8064)

In [22]:
# 32 subjects that watched 40 videos each that have 4 emotional labels per video
labels.shape

(32, 40, 4)

## Pre-Processing